In [1]:
from selenium import webdriver as wd
from urllib.request import urlopen
import time
from selenium.webdriver.common.by import By
# WebDriverWait는 Selenium 2.4.0 이후 부터 사용 가능합니다.
from selenium.webdriver.support.ui import WebDriverWait
# expected_conditions는 Selenium 2.26.0 이후 부터 사용 가능합니다.
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime as dt
import datetime
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import sys
import os
if not sys.modules.get('selenium'):
    os.system('pip3 install selenium')

In [2]:
#['000030', '000070', '000080', '000100', '000120', '000150', '000210', '000240', '000270', '000640', '000660', '000670', '000720', '000810']
#['000880', '000990', '001040', '001060', '001120', '001230', '001430', '001450', '001520', '001680', '001740', '001800', '002240', '002270']
#['002350', '002380', '002790', '002960', '003000', '003240', '003300', '003410', '003490', '003520', '003550', '003620', '003850', '003920']
#['004000', '004020', '004170', '004370', '004490', '004800', '004990', '005180', '005250', '005300', '005380', '005440', '005490', '005610']
#['005830', '005850', '005930', '005940', '006120', '006260', '006280', '006360', '006400', '006650', '006800', '006840', '007070', '007310']
#['007570', '008060', '008560', '008770', '008930', '009150', '009240', '009420', '009540', '009830', '010060', '010120', '010130', '010140']
#['010620', '010780', '010950', '011070', '011170', '011210', '011780', '011790', '012330', '012450', '012630', '012750', '014680', '014820']
#['014830', '015760', '016360', '017800', '018260', '018880', '019680', '020000', '020150', '021240', '023530', '024110', '025540', '025860']
#['026960', '027410', '028050', '028260', '028670', '029780', '032830', '033780', '033920', '034020', '034220', '034730', '035250', '036460']
#['036580', '042660', '042670', '047040', '047050', '047810', '049770', '051600', '051900', '051910', '052690', '055550', '057050', '060980']
#['064350', '064960', '066570', '068270', '069260', '069620', '069960', '071050', '071840', '073240', '078930', '079430', '079550', '086280']
#['086790', '088350', '090430', '093050', '093370', '096760', '096770', '097230', '097950', '103140', '105560', '105630', '108670', '111770']
#['114090', '115390', '120110', '128940', '138930', '139480', '145990', '161390', '161890', '170900', '185750', '192400', '192820', '204320']
#['207940', '241560', '267250', '271560', '282330', '285130', '294870', '298040']

In [3]:
def start_crawling(stock_code):    
    driver.get( 'http://www.paxnet.co.kr/' )    
    # 검색창으로 커서를 올리기 위한 명령어
    elem = driver.find_element_by_class_name("ui-autocomplete-input")
    # 검색창에 입력한 값이 있을수도 있음으로 클리어처리 한다.
    # elem.clear()
    # 종목코드를 입력한다. 이부분은 차후 함수에 변수명이 들어갈 자리이다.
    # 객체화를 시키기 위한 부분은 위에서부터 처리하도록 한다.
    elem.send_keys(stock_code)
    # 품목토론방으로 가기 위한 코드
    time.sleep(5)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'body > div.wrap > div.header > div.logo-nav-search > div.search > div.search-box > p > button'))).click()
#     driver.find_element_by_css_selector('body > div.wrap > div.header > div.logo-nav-search > div.search > div.search-box > p > button').click()
    #driver.find_element_by_css_selector('#gnbHtml > div:nth-child(1) > div > div.pan-main-tab > ul > li:nth-child(2) > a > span').click()
    # 첫번째 게시글에 들어가기
    driver.find_element_by_css_selector('#contents > div.cont-area.toggle_wrap > div.pan-main-box > div.pan-left > div.pan-real-time > ul > li:nth-child(1) > a').click()

In [4]:
def collecting(input_date):
    global collected_date
    global data 
    stime = time.time()
    idx = 0   
    while True:
    # 제목, 작성일, 글내용, 댓글 순서로 수집
        date    = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#cont-area > div.board-view > div.board-view-tit > div.writer > p.text > span.time')))
        title   = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#cont-area > div.board-view > div.board-view-tit > h1')))
        if (dt.strptime(date.text.split(' ')[0],"%Y.%m.%d").date() - dt.strptime(input_date,"%Y.%m.%d").date()).days < 0 :
            break
        text    = driver.find_element_by_class_name('board-view-cont')
        writer  = driver.find_element_by_css_selector('#cont-area > div.board-view > div.board-view-tit > div.writer > p.text > span.nickname')
    #     print(date.text,text.text)
        reply_list = []
        mord  = [date.text,title.text,writer.text,text.text]
#         reply = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#cmt-list')))
        try:
            reply = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#cmt-list')))
        # print(reply.text)
            re1 = reply.find_elements_by_class_name('cont')
            for i in re1:
                reply_list.append(i.text)
        except Exception as e:            
            pass
        mord.append(reply_list)
        mord = tuple(mord)
        data.append(mord)        
        # 다음게시글로 이동
        tr = driver.find_element_by_css_selector('#cont-area > div.prev-next > div.next > dl > dd > a').click()
        if tr == False or idx % 100== 0:
            save_csv(data,stock_code)
            data = []
        idx += 1

In [5]:
# def click_next_title():
#     # 다음 글로 넘어 가기 위한 제목 목록 
#     tr = driver.find_elements_by_css_selector('#cont-area > div.prev-next > div.next > dl > dd > a')       
#     for idx, element in enumerate(tr):
#         # 첫번째는 빈칸이다.
#         if idx==0:
#             continue        
#         try:
#             #현재 글의 위치를 찾기 위해 strong을 검색
#             td = element.find_elements_by_css_selector('td')[1].find_element_by_css_selector('strong')            
#         except Exception as e:            
#             continue

In [6]:
# 크롤러한 파일을 저장시키기.
def save_csv(jungbo, stock_code):      
    frame = pd.DataFrame(jungbo)
    with open('./paxnet_{0}.csv'.format(stock_code), 'a',encoding='utf-8') as f:
        frame.to_csv(f, header=False, index=False, encoding='utf-8')            

In [7]:
# 초기설정 위치
#크롤링 임시 그릇 -> 모니터링을 위해 글로벌 변수로 뺏음.
data = [] 
# 마감일 
deadline_date = '2017.05.23'
# 수집된 글의 일자
collected_date = set() # 지역 변수는 외부에서 꺼내보기가 쉽지 않다.
# 리스트 
stock_code_list=['000660','041960']
stock_code=''
driver = wd.Chrome('./data/chromedriver.exe')

for present_code in stock_code_list:
    stock_code=present_code
    start_crawling(stock_code)
    collecting(deadline_date)


KeyboardInterrupt: 

In [8]:
data

[('2018.12.26 14:38', '59800매수', '벤츠S550', '배당받자', []),
 ('2018.12.26 13:03', '배당', '네팔초모랑마', '주당 얼마나 줄까요?', ['1,500원 가능?', '작년에는?']),
 ('2018.12.26 11:34',
  '~~~누군가 하였다~~~',
  '님향기',
  '타락허는데 계절이있나\n망가지는데 밤낯을 가리나?\n 정도끊고\n님도끊었다....',
  []),
 ('2018.12.26 10:21',
  '배당받을까? 말까?',
  '벤츠S550',
  '60800원 매도\n\n어차피 떨어질거\n\n선물옵션 매도는\n오늘 내일 청산',
  []),
 ('2018.12.26 10:00',
  '너밖에 살게 없긴허다',
  '벤츠S550',
  '지들따라하거나 주식하지 말라한다\n삼만 찰천주 남았다',
  []),
 ('2018.12.26 09:41', '양키가 산다', '벤츠S550', '지들따라하라한다', []),
 ('2018.12.26 09:37',
  '60500원 매도 천개떨어지고 댓개 오르는 장세',
  '벤츠S550',
  '천주 팔렸어\n이게 사기지 주식시장이냐?\n그것두 개미들 안할까봐\n몇개 올리는겨~~~',
  []),
 ('2018.12.26 07:39',
  '반지하 월세방에서',
  '닭치그라',
  '희미한 형광등 불빛아래 \n중고 노트북 켜놓고 \n오른발 덜덜거리며\n호가창 뚫어지라 쳐다보다가\n오늘도 삼만원 일당목표 실패하니\n.\n싸구려 꽁초 한대물고\n멍하니 천장을 쳐다볼때\n바퀴벌레가 지나가더라.\n.\n몰빵은 무슨 몰빵\n구라빵이지 .ㅋ',
  ['ㅋㅋ 상처될라 ㅋㅋ', '저게 벵의 현실이지.ㅋ', '전세로 살아보는게\n소원 일지도.....']),
 ('2018.12.26 07:21',
  '내일 모레는 가겠지 암만.ㅋ',
  '닭치그라',
  '서울역 노숙자 쉼터로.ㅋ',
  ['지금은 주식이 아니고\n우체국 예금때라고\n그렇게 알려줘도...ㅉㅉ',
  